<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br>
<b> (Little demo) Accessibility utilities </b> <br>
Contact author(s): Andrés A. Plazas Malagón <br>
Last verified to run: 2024-08-01 <br>
LSST Science Pipelines version: Weekly 2024_16 <br>
Container Size: small

## 1. Introduction

This notebook will demonstrate:

- the use of functions to calculate color contrast to check the [Web Content Accessibility Guidelines (WCAG)](https://www.w3.org/WAI/standards-guidelines/wcag/) international standard,
- the use of the software [`Glasbey`](https://github.com/lmcinnes/glasbey) to find color-blind friendly palettes,
- the use of the software [`DaltonLens`](https://github.com/DaltonLens/DaltonLens-Python/tree/master) to simulate color vision deficiencies (e.g., deuteranomaly)

### 1.1 Package Imports

`Glasbey` and `DaltonLens` can be installed in the Rubin Science Platform by opening a new terminal and using `pip`:

- `python3 -m pip install --user daltonlens`
- `python3 -m pip install --user glasbey`

The [Python Image Library](https://omz-software.com/pythonista/docs/ios/PIL.html), PIL, is used to read a PNG image to be used by `DaltonLens`.
The [seaborn](https://seaborn.pydata.org/) package is used to display the `glasbey` color palettes. 

In [ ]:
import PIL
import numpy as np
from daltonlens import convert, simulate, generate
import matplotlib.pyplot as plt
import glasbey
import seaborn as sns

### 1.2 Define Functions and Parameters

The functions `luminance`, `contrast_ratio`, `hex_to_rgb`, and `check_contrast` can be used to calculate the relative contrast between two colors and compare them with the Web Content Accessibility Guidelines. These guidelines are detailed in the official WCAG documentation, which can be found here: [Web Content Accessibility Guidelines (WCAG) 2.1](https://www.w3.org/TR/WCAG21/#contrast-minimum)

These calculations are also performed by websites such as the [WebAIM Contrast Checker](https://webaim.org/resources/contrastchecker/). 

The function `display_color_vision_simulations` displays an original image and simulated versions for different types of color vision deficiencies using the `DaltonLens` software.

**_Function_**: `luminance` 

Calculate the relative luminance of a color based on its RGB values.

The formula for calculating the relative luminance (L) of a color is:

$L = 0.2126 \times R + 0.7152 \times G + 0.0722 \times B$

Where:
- $R$, $G$, and $B$ are the linearized values of the red, green, and blue components of the color, respectively.

The linearization process involves converting the sRGB values (which are in the range 0-255) to a linear scale:
- For sRGB values $\leq 0.03928$, linearized $R, G, B = \frac{\text{sRGB}}{12.92}$
- For sRGB values $> 0.03928$, linearized $R, G, B = \left(\frac{\text{sRGB} + 0.055}{1.055}\right)^{2.4}$

Reference: [relative luminance](https://www.w3.org/TR/WCAG21/#dfn-relative-luminance)

In [ ]:
def luminance(r, g, b):
    """
    Calculate the relative luminance of a color based on its RGB values.

    The function converts the RGB values to a luminance value according to
    the formula specified by the WCAG guidelines, which accounts for the
    human eye's sensitivity to different colors.

    Parameters
    ----------
    r : int
        The red component of the color, in the range 0-255.
    g : int
        The green component of the color, in the range 0-255.
    b : int
        The blue component of the color, in the range 0-255.

    Returns
    -------
    float
        The relative luminance of the color.
    """
    a = [r, g, b]
    for i in range(3):
        a[i] = a[i] / 255.0
        a[i] = a[i] / 12.92 if a[i] <= 0.03928 else ((a[i] + 0.055) / 1.055) ** 2.4
    return a[0] * 0.2126 + a[1] * 0.7152 + a[2] * 0.0722

**_Function_**: `contrast_ratio`

Compute the contrast ratio between two colors based on their luminance values.

The contrast ratio between two colors is calculated as:

$\text{Contrast Ratio} = \frac{L1 + 0.05}{L2 + 0.05}$

Where:
- $L1$ and $L2$ are the relative luminance values of the two colors.
- $L1$ is the luminance of the lighter color.
- $L2$ is the luminance of the darker color.

Reference: [contrast ratio](https://www.w3.org/TR/WCAG21/#dfn-contrast-ratio).


In [ ]:
def contrast_ratio(color1, color2):
    """
    Calculate the contrast ratio between two colors.

    The contrast ratio is computed based on the luminance values of the two colors,
    following the WCAG guidelines. It is used to determine the readability of text
    or the visibility of UI components against background colors.

    Parameters
    ----------
    color1 : tuple
        A tuple representing the RGB values of the first color.
    color2 : tuple
        A tuple representing the RGB values of the second color.

    Returns
    -------
    float
        The contrast ratio between the two colors.
    """
    l1 = luminance(*color1)
    l2 = luminance(*color2)
    if l1 > l2:
        return (l1 + 0.05) / (l2 + 0.05)
    else:
        return (l2 + 0.05) / (l1 + 0.05)

**_Function_**: `hex_to_rgb`

Convert hexadecimal color codes to RGB tuples.

In [ ]:
def hex_to_rgb(hex_color):
    """
    Convert a hexadecimal color code to an RGB tuple.

    This function takes a hexadecimal color string and converts it into a
    tuple of integers representing the red, green, and blue components of the color.

    Parameters
    ----------
    hex_color : str
        A string representing the hexadecimal color code, e.g., '#RRGGBB'.

    Returns
    -------
    tuple
        A tuple of three integers (R, G, B) representing the color in RGB format.
    """
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

**_Function_**: `check_contrast`

Evaluate the contrast ratio against WCAG guidelines for different text and UI component sizes.

The Web Content Accessibility Guidelines (WCAG) 2.1 provide the following standards for contrast ratios:

- **Normal Text (AA)**: A contrast ratio of at least 4.5:1.
- **Large Text (AA)**: A contrast ratio of at least 3:1.
- **Graphics and User Interface Components (AA)**: A contrast ratio of at least 3:1.
- **Normal Text (AAA)**: A contrast ratio of at least 7:1.
- **Large Text (AAA)**: A contrast ratio of at least 4.5:1.

Reference: [WCGA 2.1 standards](https://www.w3.org/TR/WCAG21/#contrast-minimum)

In [ ]:
def check_contrast(ratio):
    """
    Evaluate the contrast ratio against WCAG guidelines for different text and UI component sizes.

    The function checks whether the given contrast ratio meets the WCAG requirements
    for normal text, large text, and graphical UI components under different compliance levels.

    Parameters
    ----------
    ratio : float
        The contrast ratio to evaluate.

    Returns
    -------
    dict
        A dictionary with the results for different WCAG criteria, indicating 'Pass' or 'Fail'.
    """
    results = {
        "Normal Text AA": ratio >= 4.5,
        "Large Text AA": ratio >= 3,
        "Graphics/UI Components AA": ratio >= 3,
        "Normal Text AAA": ratio >= 7,
        "Large Text AAA": ratio >= 4.5
    }
    return results

**_Function_**: `display_color_vision_simulations`

Use `DaltonLens` to display an original image and simulated versions for different types of color vision deficiencies.

In [ ]:
def display_color_vision_simulations(image, simulator, color_anomalies):
    """
    Display the original image and simulated versions for different types of color vision deficiencies.

    Parameters
    ----------
    image: `np.array`
        The original image to be displayed.

    simulator: `DaltonLens` obejct
        `DaltonLens` simulator.

    color_anomalies: `list`
        List of color vision anomalies.
    """
    fig, axs = plt.subplots(2, 2, figsize=(12, 12))

    for idx, (title, deficiency) in enumerate(color_anomalies):
        ax = axs[idx // 2, idx % 2]
        
        if deficiency is None:
            # Show the original image
            display_im = image
        else:
            # Apply the simulator to the input image to simulate the color anomaly
            display_im = simulator.simulate_cvd(image, deficiency, severity=1.0)
        
        ax.imshow(display_im)
        ax.set_title(title)
        ax.axis('off')  # Hide the axis

    plt.tight_layout()
    plt.show()

## 2 Color contrast calculation example

In [ ]:
color1 = "#3D7555"
color2 = "#0400FB"
rgb1 = hex_to_rgb(color1)
rgb2 = hex_to_rgb(color2)

ratio = contrast_ratio(rgb1, rgb2)
results = check_contrast(ratio)

print(f"The contrast ratio between {color1} and {color2} is: {ratio:.2f}")
print("Pass/Fail Criteria:")
for criteria, passed in results.items():
    print(f"{criteria}: {'Pass' if passed else 'Fail'}") 

## 3. Demonstration of the `glasbey` software

The `glasbey` library enables the creation of color palettes optimized for categorical data, based on techniques from the paper ["Colour Displays for Categorical Images" by Glasbey, Heijden, Toh, and Gray](https://onlinelibrary.wiley.com/doi/abs/10.1002/col.20327). This library also provides features designed to make these palettes more distinguishable for individuals with various degrees of color vision deficiency.

Reference: [Glasbey color-blind safe palettes](https://glasbey.readthedocs.io/en/latest/color_vision_deficiency.html)

The following example follows the `glasbey` [documentation](https://glasbey.readthedocs.io/en/latest/color_vision_deficiency.html)

In [ ]:
sns.set()
palette_glasbey = glasbey.create_palette(palette_size=12, colorblind_safe=True, cvd_severity=100)
sns.palplot(palette_glasbey)

## 4. Demonstration of the `DaltonLens` Software

There are several open-source simulators for color deficiencies, but some of them are inaccurate.  The following website does a good comparison between them:

- [Review of Open Source Color Blindness Simulations](https://daltonlens.org/opensource-cvd-simulation/)

The authors provide the software `DaltonLens` that uses more accurate models: 

- [DaltonLens repository](https://github.com/DaltonLens/DaltonLens-Python/tree/master)
- [Understanding the color-vision deficiency simulator implemented in `DaltonLens`](https://daltonlens.org/understanding-cvd-simulation/)

Define the simulator: `Simulator_AutoSelect()` automatically selects the most suitable algorithm based on the type of color vision deficiency and its severity. For tritan simulations, it uses the model by [Brettel, Viénot, and Molon (1997)](https://pubmed.ncbi.nlm.nih.gov/9316278/). For protanomaly and deuteranomaly with severity levels less than 1, the [Machado, Oliveira, and Fernandes (2009)](https://pubmed.ncbi.nlm.nih.gov/19834201/) model is chosen. For complete protanopia and deuteranopia (severity level equal to 1), the [Viénot, Brettel, and Mollon (1999)](https://onlinelibrary.wiley.com/doi/10.1002/(SICI)1520-6378(199908)24:4%3C243::AID-COL5%3E3.0.CO;2-3) model is utilized. This selection process ensures the use of the most accurate simulation technique for the specified conditions.

More details can be found in the file `simulate.py` from the `DaltonLens` code:

**Machado 2009**:  
A model proposed by Machado, Oliveira, and Fernandes (2009) in their paper *"A physiologically-based model for simulation of color vision deficiency"*. This model is similar to Brettel1997 for simulating dichromacy and uses it as a reference to scale parameters. It is capable of simulating various severity levels by shifting the peak wavelength for a specific cone, which is considered a more robust method than simply interpolating with the original image. However, it is noted that this model does not perform well for tritanopia.

**Brettel & Molon, 1997**:  
An algorithm developed by Brettel, Viénot, and Mollon (1997) detailed in the paper *"Computerized simulation of color appearance for dichromats"*. This model is somewhat more complex than the one by Viénot & Brettel & Mollon (1999) but is particularly effective for simulating tritanopia. It is considered one of the most reliable references in the literature for this purpose.

**Vienot, 1999**:  
An algorithm described by Viénot, Brettel, and Mollon (1999) in their study *"Digital video colourmaps for checking the legibility of displays by dichromats."* This model is recommended for simulating protanopia and deuteranopia, although it is not accurate for tritanopia.

In [ ]:
simulator = simulate.Simulator_AutoSelect()

Define the color deficiencies.

In [ ]:
color_anomalies = [
    ("Original", None), 
    ("Protanomaly", simulate.Deficiency.PROTAN), 
    ("Deuteranomaly", simulate.Deficiency.DEUTAN), 
    ("Tritanomaly", simulate.Deficiency.TRITAN)
]

Read an image from file.

In [ ]:
file = "./figures/ucd-lc-portal-screenshot.png"
im1 = np.asarray(PIL.Image.open(file).convert('RGB'))

Create the plots. 

In [ ]:
display_color_vision_simulations(im1, simulator, color_anomalies)

Read another image, consisting of light curves from DP0.2 tutorial `DP02_07b_Variable_Star_Lightcurves`

In [ ]:
file = "./figures/light_curves_dalton_lens_demo.png"
im2 = np.asarray(PIL.Image.open(file).convert('RGB'))

Create the plots. 

In [ ]:
display_color_vision_simulations(im2, simulator, color_anomalies)